In [ ]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, DoubleType, StringType,NumericType, LongType, StringType, DateType
import pandas as pd
import numpy as np
from pyspark import SparkContext
from pyspark.sql.functions import lit
import pyspark.sql.functions as func


In [ ]:
#sc=SparkContext('Unique ID','Patient Name','Vaccine Type','Date of Birth','Date of Vaccination')

In [ ]:
spark = SparkSession.builder.appName("Vaccination").getOrCreate()

In [ ]:
df_AUS = pd.read_excel("/content/drive/MyDrive/Incubyte/AUS.xlsx")
df_IND = pd.read_csv("/content/drive/MyDrive/Incubyte/IND.csv")
df_USA = pd.read_csv("/content/drive/MyDrive/Incubyte/USA.csv")

#print(df_AUS.head())
# print(df_IND.head())
# print(df_USA.head())

schema_AUS = StructType([StructField("Unique ID", LongType(), True), StructField("Patient Name", StringType(), True), StructField("Vaccine Type", StringType(), True),StructField("Date of Birth", DateType(), True),StructField("Date of Vaccination",StringType(), False)])
schema_IND = StructType([StructField("ID", LongType(), True), StructField("Name", StringType(), True), StructField("DOB", StringType(), True),StructField("VaccinationType", StringType(), True),StructField("VaccinationDate", StringType(), True),StructField("Free or Paid", StringType(), True)])
schema_USA = StructType([StructField("ID", LongType(), True), StructField("Name", StringType(), True), StructField("VaccinationType", StringType(), True),StructField("VaccinationDate", StringType(), True)])

df_AUS = spark.createDataFrame(df_AUS,schema=schema_AUS)
df_IND = spark.createDataFrame(df_IND,schema=schema_IND)
df_USA = spark.createDataFrame(df_USA,schema=schema_USA)


In [ ]:
df_IND

DataFrame[ID: bigint, Name: string, DOB: string, VaccinationType: string, VaccinationDate: string, Free or Paid: string]

In [ ]:
df_USA.head(5)

[Row(ID=1, Name='Sam', VaccinationType='EFG', VaccinationDate='6152022'),
 Row(ID=2, Name='John', VaccinationType='XYZ', VaccinationDate='1052022'),
 Row(ID=3, Name='Mike', VaccinationType='ABC', VaccinationDate='12282021')]

In [ ]:
 df_AUS.show()

+---------+------------+------------+-------------+--------------------+
|Unique ID|Patient Name|Vaccine Type|Date of Birth| Date of Vaccination|
+---------+------------+------------+-------------+--------------------+
|        1|        Mike|         LMN|         null|java.util.Gregori...|
|        2|   Jonnathan|         XYZ|   1997-12-13|          2021-13-13|
|        3|    Cristina|         ABC|   1998-03-12|java.util.Gregori...|
+---------+------------+------------+-------------+--------------------+



In [ ]:
 #Matching the columns
 df_IND = df_IND.drop( "DOB", "Free or Paid")
 df_AUS=df_AUS.drop("Date of Birth")


In [ ]:
#Adding country column
df_AUS = df_AUS.withColumn("country" ,lit("Australia"))
df_IND = df_IND.withColumn("country" ,lit("INDIA"))
df_USA = df_USA.withColumn("country" ,lit("USA"))

In [ ]:
df_AUS.head()

Row(Unique ID=1, Patient Name='Mike', Vaccine Type='LMN', Date of Vaccination='java.util.GregorianCalendar[time=?,areFieldsSet=false,areAllFieldsSet=false,lenient=true,zone=sun.util.calendar.ZoneInfo[id="Etc/UTC",offset=0,dstSavings=0,useDaylight=false,transitions=0,lastRule=null],firstDayOfWeek=1,minimalDaysInFirstWeek=1,ERA=?,YEAR=2022,MONTH=4,WEEK_OF_YEAR=?,WEEK_OF_MONTH=?,DAY_OF_MONTH=11,DAY_OF_YEAR=?,DAY_OF_WEEK=?,DAY_OF_WEEK_IN_MONTH=?,AM_PM=0,HOUR=0,HOUR_OF_DAY=0,MINUTE=0,SECOND=0,MILLISECOND=0,ZONE_OFFSET=?,DST_OFFSET=?]', country='Australia')

In [ ]:
#Renaming the column
df_AUS = df_AUS.withColumnRenamed('Unique ID', 'ID')
df_AUS = df_AUS.withColumnRenamed('Patient Name', 'Name')
df_AUS = df_AUS.withColumnRenamed('Vaccine Type', 'VaccinationType')
df_AUS = df_AUS.withColumnRenamed('Date of Vaccination', 'VaccinationDate')

df_AUS.head()

Row(ID=1, Name='Mike', VaccinationType='LMN', VaccinationDate='java.util.GregorianCalendar[time=?,areFieldsSet=false,areAllFieldsSet=false,lenient=true,zone=sun.util.calendar.ZoneInfo[id="Etc/UTC",offset=0,dstSavings=0,useDaylight=false,transitions=0,lastRule=null],firstDayOfWeek=1,minimalDaysInFirstWeek=1,ERA=?,YEAR=2022,MONTH=4,WEEK_OF_YEAR=?,WEEK_OF_MONTH=?,DAY_OF_MONTH=11,DAY_OF_YEAR=?,DAY_OF_WEEK=?,DAY_OF_WEEK_IN_MONTH=?,AM_PM=0,HOUR=0,HOUR_OF_DAY=0,MINUTE=0,SECOND=0,MILLISECOND=0,ZONE_OFFSET=?,DST_OFFSET=?]', country='Australia')

In [ ]:
# Mergining data

df_ALL = df_AUS.unionByName(df_IND)
df_ALL = df_ALL.unionByName(df_USA)

In [ ]:
df_ALL.head(20)

[Row(ID=1, Name='Mike', VaccinationType='LMN', VaccinationDate='java.util.GregorianCalendar[time=?,areFieldsSet=false,areAllFieldsSet=false,lenient=true,zone=sun.util.calendar.ZoneInfo[id="Etc/UTC",offset=0,dstSavings=0,useDaylight=false,transitions=0,lastRule=null],firstDayOfWeek=1,minimalDaysInFirstWeek=1,ERA=?,YEAR=2022,MONTH=4,WEEK_OF_YEAR=?,WEEK_OF_MONTH=?,DAY_OF_MONTH=11,DAY_OF_YEAR=?,DAY_OF_WEEK=?,DAY_OF_WEEK_IN_MONTH=?,AM_PM=0,HOUR=0,HOUR_OF_DAY=0,MINUTE=0,SECOND=0,MILLISECOND=0,ZONE_OFFSET=?,DST_OFFSET=?]', country='Australia'),
 Row(ID=2, Name='Jonnathan', VaccinationType='XYZ', VaccinationDate='2021-13-13', country='Australia'),
 Row(ID=3, Name='Cristina', VaccinationType='ABC', VaccinationDate='java.util.GregorianCalendar[time=?,areFieldsSet=false,areAllFieldsSet=false,lenient=true,zone=sun.util.calendar.ZoneInfo[id="Etc/UTC",offset=0,dstSavings=0,useDaylight=false,transitions=0,lastRule=null],firstDayOfWeek=1,minimalDaysInFirstWeek=1,ERA=?,YEAR=2022,MONTH=2,WEEK_OF_YEAR=?,

In [ ]:
#Removing dulicate
df_ALL = df_ALL.dropDuplicates()

In [ ]:
df_vacinnation=df_ALL.groupBy("country").count().toDF("Country","TotalVaccination")

In [ ]:
df_vacinnation.show()

+---------+----------------+
|  Country|TotalVaccination|
+---------+----------------+
|      USA|               3|
|    INDIA|               3|
|Australia|               3|
+---------+----------------+



In [ ]:
df_vacinnation = df_vacinnation.withColumn("TotalPopulationEligibleForVaccination" ,lit(100))
df_vacinnation = df_vacinnation.withColumn("Percentage Vaccinated" ,df_vacinnation["TotalVaccination"]/df_vacinnation["TotalPopulationEligibleForVaccination"]*100)
df_vacinnation = df_vacinnation.withColumn("Percentage Vaccinated" ,df_vacinnation["TotalVaccination"]/df_vacinnation["TotalPopulationEligibleForVaccination"]*100)

In [ ]:
df_vacinnation.show()

+---------+----------------+-------------------------------------+---------------------+
|  Country|TotalVaccination|TotalPopulationEligibleForVaccination|Percentage Vaccinated|
+---------+----------------+-------------------------------------+---------------------+
|      USA|               3|                                  100|                  3.0|
|    INDIA|               3|                                  100|                  3.0|
|Australia|               3|                                  100|                  3.0|
+---------+----------------+-------------------------------------+---------------------+



In [ ]:
df_country_type=df_ALL.groupBy("country","VaccinationType").count().toDF("Country","VaccinationType","TotalVaccination").sort("country")

In [ ]:
df_country_type.show()

+---------+---------------+----------------+
|  Country|VaccinationType|TotalVaccination|
+---------+---------------+----------------+
|Australia|            ABC|               1|
|Australia|            XYZ|               1|
|Australia|            LMN|               1|
|    INDIA|            ABC|               2|
|    INDIA|            XYZ|               1|
|      USA|            XYZ|               1|
|      USA|            ABC|               1|
|      USA|            EFG|               1|
+---------+---------------+----------------+



In [ ]:
df_Type=df_ALL.groupBy("VaccinationType").count().toDF('VaccinationType',"TotalVaccination")
total_vaccination_type=df_Type.agg({'TotalVaccination': 'sum'}).collect()
df_Type = df_Type.withColumn("Percentage Contribution" ,func.round(df_Type["TotalVaccination"]/total_vaccination_type[0][0]*100,2))
df_Type.show()

+---------------+----------------+-----------------------+
|VaccinationType|TotalVaccination|Percentage Contribution|
+---------------+----------------+-----------------------+
|            EFG|               1|                  11.11|
|            LMN|               1|                  11.11|
|            XYZ|               3|                  33.33|
|            ABC|               4|                  44.44|
+---------------+----------------+-----------------------+



In [ ]:
df_contribution=df_ALL.groupBy("country").count().toDF("Country","TotalVaccination")
#total_vaccination=df_contribution.select(sum('TotalVaccination')).collect()
#total_vaccination
total_vaccination=df_contribution.agg({'TotalVaccination': 'sum'}).collect()
#df_contribution.schema
df_contribution = df_contribution.withColumn("Percentage Contribution" ,func.round(df_contribution["TotalVaccination"]/total_vaccination[0][0]*100,2))
df_contribution.show()

+---------+----------------+-----------------------+
|  Country|TotalVaccination|Percentage Contribution|
+---------+----------------+-----------------------+
|      USA|               3|                  33.33|
|    INDIA|               3|                  33.33|
|Australia|               3|                  33.33|
+---------+----------------+-----------------------+

